In [108]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import csv

In [109]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [110]:
#check if there is an intersection or if the boxes do not overlap
def checkForItersection(x1,x2,x3,x4,y1,y2,y3,y4):
    intersection = True
    if(x4 < x1):
        intersection = False
    if(x2 < x3):
        intersection = False
    if(y4 < y1):
        intersection = False
    if(y2 < y3):
        intersection = False
    return intersection

In [215]:
#compute the itersection of all of the boxes
#returns the total size of all intersections

def findIntersection(listOfGT, listOfResults):
    
    sizeOfGT = len(listOfGT)
    sizeOfResults = len(listOfResults)
    
    totalIntersection = 0
    
    for i in range(0, sizeOfGT):
        for j in range(0, sizeOfResults):
            x1 = listOfGT[i][0]
            y1 = listOfGT[i][1]
            x2 = listOfGT[i][2]
            y2 = listOfGT[i][3]
            
            x3 = listOfResults[j][0]
            y3 = listOfResults[j][1]
            x4 = listOfResults[j][2]
            y4 = listOfResults[j][3]
            intersection = checkForItersection(x1,x2,x3,x4,y1,y2,y3,y4)
            if(intersection == False):
                continue
            x5 = max(x1, x3)
            y5 = max(y1, y3)
            x6 = min(x2, x4)
            y6 = min(y2, y4)
            
            x7 = int(x6)-int(x5)
            y7 = int(y6)-int(y5)
            sizeOfInt = int(x7)*int(y7)
            
            totalIntersection = totalIntersection + sizeOfInt
    if(totalIntersection < 0):
        totalIntersection = 0
    #print(totalIntersection)
    return(totalIntersection)

In [155]:
#find the total size of all of the boxes for a single image
def findSizeOfWindows(listOfWindows):
    numberOfWindows = len(listOfWindows)
    
    totalSize = 0
    
    for i in range (0, numberOfWindows):
        x1 = listOfWindows[i][0]
        y1 = listOfWindows[i][1]
        x2 = listOfWindows[i][2]
        y2 = listOfWindows[i][3]
        
        x3 = int(x2)-int(x1)
        y3 = int(y2)-int(y1)
        sizeOfInt = x3*y3
        totalSize = totalSize + sizeOfInt
    #print(totalSize)
    return totalSize

In [196]:
#get the values for a certain image number from the given csv file and return just the pertinant rows
def getCSVVal(inputString, imageString, r1, r2, r3, r4):
    
    referenceToCSV = open(inputString)
    csvReader = csv.reader(referenceToCSV)
    
    rowList = []
    resultList = []
    for row in csvReader:
        test = row[0]
        if(test == imageString):
            rowList.append(row)
    for row in rowList:
        result = [row[r1], row[r2], row[r3], row[r4]]
        resultList.append(result)
    return(resultList)

In [244]:
#get the filepaths for the two csv files
gtString = '_test_labels_minimized.csv'
resultString = '_predicted_boxes_val.csv'

diceList = []

#add labels that will be placed in the CSV
diceList.append(['ImageNumber', 'DICE'])

#for each of the images in the csv we will check to find all of the boxes are found in that specific image
#in both the ground truth and also in the results found by the algorithm
#it will then compute the DICE coefficient as shown in the project description
#before storing it in a list and saving that list to a csv file
for i in range(12001, 15000):
    
    imageString = '000' + str(i) + '.jpg'
    
    resultList1 = getCSVVal(gtString, imageString, 4,5,6,7)
    resultList2 = getCSVVal(resultString, imageString, 4,5,6,7)
    
    truePos = findIntersection(resultList1, resultList2)
    
    falsePos = findSizeOfWindows(resultList1)

    falseNeg = findSizeOfWindows(resultList2)
    
    if(falsePos == 0 and falsePos == 0):
        continue
    
    dice = (2*truePos)/(2*truePos + falseNeg + falsePos)
#     if(dice == 0):
#         continue
    
    testList = []
    testList.append(imageString)
    testList.append(dice)
    
    diceList.append(testList)
print(diceList)

with open("dice_results1.csv",'w') as resultFile:
    wr = csv.writer(resultFile, lineterminator='\n')
    wr.writerows(diceList)

[['ImageNumber', 'DICE'], ['00012001.jpg', 0.3691523077483516], ['00012002.jpg', 0.43570892514662146], ['00012003.jpg', 0.10314731552499339], ['00012004.jpg', 0.2852848101265823], ['00012005.jpg', 0.07013340593520283], ['00012006.jpg', 0.0], ['00012007.jpg', 0.0], ['00012008.jpg', 0.41173567051515114], ['00012009.jpg', 0.0], ['00012010.jpg', 0.46913992404957644], ['00012011.jpg', 0.4775612448520358], ['00012012.jpg', 0.13686399450559933], ['00012013.jpg', 0.4836852207293666], ['00012014.jpg', 0.46944645939118035], ['00012015.jpg', 0.0], ['00012016.jpg', 0.06581458562257822], ['00012017.jpg', 0.4830645416811888], ['00012018.jpg', 0.016881089519298058], ['00012019.jpg', 0.0], ['00012020.jpg', 0.058890722172864576], ['00012021.jpg', 0.0], ['00012022.jpg', 0.46816603921185157], ['00012023.jpg', 0.4324297634918284], ['00012024.jpg', 0.16531642597158624], ['00012025.jpg', 0.023635254650808173], ['00012026.jpg', 0.18874172185430463], ['00012027.jpg', 0.0], ['00012028.jpg', 0.5442421734933113]